# 🎣 낚시성기사분류기 만들기
> 기사의 제목과 본문을 입력으로 줬을때, `제목과 본문사이 관계가 아래 6가지에 포함된다면 낚시성기사로 판단 `

#### 실습 목차

1. Dataset & Tokenizing
  * 1-1. news_dataset class 정의
  * 1-2. load_Data, construct_tokenized_dataset 함수 정의
  * 1-3. prepare_dataset 함수 정의
2. Model & Trainer
  * 2-1. compute_metrics 함수 정의
  * 2-2. load_tokenizer_and_model_for_train 함수 정의
  * 2-3. load_trainer_for_train 함수 정의
  * 2-4. train 함수 정의
  * 2-5. arguments 지정 및 학습 진행
3. Inference & Evaluation
  * 3-1. load_model_for_inference 함수 정의
  * 3-2. inference 함수 정의
  * 3-3. infer_and_eval 함수 정의

In [1]:
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1gucFY-P9a1TzdV8Xb-OwVD4TpPy4iyJX' -O train.csv
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1J05RaqfknDzTObofL7OyiSmT0B8rX0gZ' -O test.csv

In [2]:
import os
import torch
import random
import numpy as np
import pandas as pd
import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
)

from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.optimization import get_cosine_with_hard_restarts_schedule_with_warmup

2024-08-28 11:41:25.565359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 11:41:25.642958: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 11:41:25.988176: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2024-08-28 11:41:25.988236: W tensorflow/compiler/xl

In [3]:
class news_dataset(Dataset):
    def __init__(self, news_dataset, labels):
        self.dataset = news_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key : val[idx].clone().detach() for key, val in self.dataset.items()
        }

        item['labels'] = torch.tensor(self.labels[idx])
        
        return item
    
    def __len__(self):
        return len(self.labels)

In [4]:
def load_data(dataset_dir):
    """csv file을 dataframe으로 load"""
    dataset = pd.read_csv(dataset_dir)[:500]
    print("dataframe 의 형태")
    print(dataset.head())
    return dataset


def construct_tokenized_dataset(dataset, tokenizer, max_length):
    """[뉴스제목 + [SEP] + 뉴스본문]형태로 토크나이징"""
    concat_entity = []
    for title, body in zip(dataset["newsTitle"],dataset["newsContent"]):
        total = str(title) + "[SEP]" + str(body)
        concat_entity.append(total)

    print("tokenizer 에 들어가는 데이터 형태")
    print(concat_entity[:5])

    ## tokenizer가 입력 텍스트를 토큰화할 때, 패딩된 토큰을 구분하기 위한 attention_mask도 함께 생성됩니다. attention_mask는 시퀀스에서 실제 토큰 위치는 1로, 패딩된 토큰 위치는 0으로 표시하는 텐서입니다.
    ## 이후, 이 attention_mask는 news_dataset 클래스의 __getitem__ 메서드를 통해 dataloader로 전달되며, 최종적으로 inference 함수 내의 모델에 입력으로 사용됩니다.
    tokenized_senetences = tokenizer(
        concat_entity,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = max_length,
        add_special_tokens = True,
        return_token_type_ids=False, # BERT 이후 모델(RoBERTa 등) 사용할때 On
    )
    print("tokenizing 된 데이터 형태")
    print(tokenized_senetences[:5])
    
    return tokenized_senetences


def prepare_dataset(dataset_dir, tokenizer,max_len):
    """학습(train)과 평가(test)를 위한 데이터셋을 준비"""
    # load_data
    train_dataset = load_data(os.path.join(dataset_dir, "train.csv"))
    test_dataset = load_data(os.path.join(dataset_dir, "test.csv"))

    # split train / validation = 7.5 : 2.5
    train_dataset, val_dataset = train_test_split(train_dataset,test_size=0.25,random_state=42,stratify=train_dataset['label'])

    # split label
    train_label = train_dataset['label'].values
    val_label = val_dataset['label'].values
    test_label = test_dataset['label'].values

    # tokenizing dataset
    tokenized_train = construct_tokenized_dataset(train_dataset, tokenizer, max_len)
    tokenized_val = construct_tokenized_dataset(val_dataset, tokenizer, max_len)
    tokenized_test = construct_tokenized_dataset(test_dataset, tokenizer, max_len)
    print("--- tokenizing Done ---")

    # make dataset for pytorch.
    news_train_dataset = news_dataset(tokenized_train, train_label)
    news_val_dataset = news_dataset(tokenized_val, val_label)
    news_test_dataset = news_dataset(tokenized_test, test_label)
    print("--- dataset class Done ---")

    return news_train_dataset , news_val_dataset, news_test_dataset , test_dataset

In [5]:
def compute_metrics(pred):
    """validation을 위한 metrics function"""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1) ## -1은 마지막 차원을 대상(축)으로 한다는 뜻

    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)

    # calculate f1 score using sklearn's function
    f1 = f1_score(labels, preds, average='micro')

    return {
        "accuracy": acc,
        "f1": f1,
    }

In [6]:
def load_tokenizer_and_model_for_train(args):
    """학습(train)을 위한 사전학습(pretrained) 토크나이저와 모델을 huggingface에서 load"""
    # load model and tokenizer
    MODEL_NAME = args.model_name
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # setting model hyperparameter
    model_config = AutoConfig.from_pretrained(MODEL_NAME)
    model_config.num_labels = 2
    print(model_config)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, config=model_config
    )
    print("--- Modeling Done ---")
    return tokenizer , model

In [7]:
def load_trainer_for_train(args, model, news_train_dataset, news_val_dataset):
    """학습(train)을 위한 huggingface trainer 설정"""
    training_args = TrainingArguments(
        output_dir=args.save_path + "results",  # output directory
        save_total_limit=args.save_limit,  # number of total save model.
        save_steps=args.save_step,  # model saving step.
        num_train_epochs=args.epochs,  # total number of training epochs
        learning_rate=args.lr,  # learning_rate
        per_device_train_batch_size=args.batch_size,  # batch size per device during training
        per_device_eval_batch_size=2,  # batch size for evaluation
        warmup_steps=args.warmup_steps,  # number of warmup steps for learning rate scheduler
        weight_decay=args.weight_decay,  # strength of weight decay
        logging_dir=args.save_path + "logs",  # directory for storing logs
        logging_steps=args.logging_step,  # log saving step.
        evaluation_strategy="steps",  # evaluation strategy to adopt during training
            # `no`: No evaluation during training.
            # `steps`: Evaluate every `eval_steps`.
            # `epoch`: Evaluate every end of epoch.
        eval_steps=args.eval_step,  # evaluation step.
        load_best_model_at_end=True,
    )

    ## Add callback & optimizer & scheduler
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=3, early_stopping_threshold=0.001
    )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=args.lr,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=args.weight_decay,
        amsgrad=False,
    )
    print("--- Set training arguments Done ---")

    trainer = Trainer(
        model=model,  # the instantiated 🤗 Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=news_train_dataset,  # training dataset
        eval_dataset=news_val_dataset,  # evaluation dataset
        compute_metrics=compute_metrics,  # define metrics function
        callbacks=[MyCallback],
        optimizers=(
            optimizer,
            get_cosine_with_hard_restarts_schedule_with_warmup(
                    optimizer,
                    num_warmup_steps=args.warmup_steps,
                    num_training_steps=len(news_train_dataset) * args.epochs,
            ),
        ),
    )
    print("--- Set Trainer Done ---")

    return trainer


In [8]:
def train(args):
    """모델을 학습(train)하고 best model을 저장"""
    # fix a seed
    pl.seed_everything(seed=42, workers=False)

    # set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("device:", device)

    # set model and tokenizer
    tokenizer , model = load_tokenizer_and_model_for_train(args)
    model.to(device)

    # set data
    news_train_dataset, news_val_dataset, news_test_dataset, test_dataset = prepare_dataset(args.dataset_dir, tokenizer, args.max_len)

    # set trainer
    trainer = load_trainer_for_train(args, model, news_train_dataset, news_val_dataset)

    # train model
    print("--- Start train ---")
    trainer.train()
    print("--- Finish train ---")
    model.save_pretrained("./best_model")


In [9]:
class args():
  """학습(train)과 추론(infer)에 사용되는 arguments 관리하는 class"""
  dataset_dir = "./"
  model_type = "roberta" # 다른 모델 사용 가능 e.g) "bert" , "electra" ···
  model_name = "klue/roberta-large" # 다른 모델 사용 가능 e.g) "klue/bert-base" , "monologg/koelectra-base-finetuned-nsmc" ···
  save_path = "./"
  save_step = 200
  logging_step = 200
  eval_step = 100
  save_limit = 5
  seed = 42
  epochs = 1 # 10
  batch_size = 8 # 메모리 상황에 맞게 조절 e.g) 16 or 32
  max_len = 256
  lr = 3e-5
  weight_decay = 0.01
  warmup_steps = 300
  scheduler = "linear"
  model_dir = "./best_model" #추론 시, 저장된 모델 불러오는 경로 설정

train(args)

Seed set to 42


device: cuda:0


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- Modeling Done ---
dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory                        newsTitle  \
0       12529           경제           금융/증권  김종민 대전산단협회장 “산단 발전과 인식변화 위해 노력”   
1        4815           경제              유통     힐튼 가든 인, 내달 1일 루프톱 야외 수영장 개장   
2       11556           경제           유통/쇼핑                    이마트에서 겨울패션 충전   
3       11798           경제           유통/쇼핑      새 통큰상품 기대하세요…롯데마트 창립17주년 행사   
4       15381           경제            대덕특구              추석 열차 승차권 10~11일 예매   

                                         newsContent  label  
0  “대전산업단지 발전과 인식 변화를 위해 최선의 노력을 다하겠다.” 제14대 대전산업...      1  
1  힐튼 가든 인 서울 강남은 다음달 1일부터 루프톱 야외 수영장을 오픈, 온수풀을 가...      0  
2  이마트가 지속되는 한파에 끝이 보이지 않는 겨울을 이기기 위한 비법으로 겨울 패션 ...      1  
3  롯데마트는 창립 17주년을 맞아 다양한 '통큰 상품'을 새롭게 출시한다고 26일 밝...      1  
4  올해 추석 연휴(9월 9~14일) 열차 승차권(좌석지정 승차권) 예매가 오는 10일...      1  
dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory  \
0       21851           경제              산업   
1        

/home/pervinco/.local/lib/python3.8/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


--- Finish train ---


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import pandas as pd
import torch
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [11]:
def load_model_for_inference(args):
    """추론(infer)에 필요한 모델과 토크나이저 load """
    # load tokenizer
    Tokenizer_NAME = args.model_name
    tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)

    ## load my model
    model = AutoModelForSequenceClassification.from_pretrained(args.model_dir)

    return tokenizer, model

In [12]:
def inference(args, model, tokenized_sent, device):
    """학습된(trained) 모델을 통해 결과를 추론하는 function"""
    dataloader = DataLoader(tokenized_sent, batch_size=args.batch_size, shuffle=False)
    model.eval()
    output_pred = []
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data["input_ids"].to(device),
                attention_mask=data["attention_mask"].to(device),
            )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)

        output_pred.append(result)
    return (np.concatenate(output_pred).tolist(),)

In [15]:
def infer_and_eval(args):
    """학습된 모델로 추론(infer)한 후에 예측한 결과(pred)를 평가(eval)"""
    # set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # set model & tokenizer
    tokenizer, model = load_model_for_inference(args)
    model.to(device)

    # set data
    news_train_dataset, news_val_dataset, news_test_dataset, test_dataset = prepare_dataset(args.dataset_dir,tokenizer,args.max_len)

    # predict answer
    pred_answer = inference(args, model, news_test_dataset, device)  # model에서 class 추론
    print("--- Prediction done ---")

    # evaluate between label and prediction
    labels = test_dataset['label'].values
    pred = pred_answer[0]

    acc = accuracy_score(labels, pred)
    f1 = f1_score(labels, pred, average='macro')
    print(f" ----- accuracy:{acc * 100:.1f}% -----")
    print(f"----- f1_score(macro): {f1 * 100:.1f}% ------")

    # make csv file with predicted answer
    output = pd.DataFrame(
        {
            "title": test_dataset["newsTitle"],
            "cleanBody": test_dataset["newsContent"],
            "clickbaitClass": pred,
        }
    )

    # 최종적으로 완성된 예측한 라벨 csv 파일 형태로 저장.
    result_path = "./prediction/"
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(
        os.path.join(result_path,"result.csv"), index=False
    )
    print("--- Save result ---")
    return output

In [16]:
output_df = infer_and_eval(args)
output_df.head(10)

dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory                        newsTitle  \
0       12529           경제           금융/증권  김종민 대전산단협회장 “산단 발전과 인식변화 위해 노력”   
1        4815           경제              유통     힐튼 가든 인, 내달 1일 루프톱 야외 수영장 개장   
2       11556           경제           유통/쇼핑                    이마트에서 겨울패션 충전   
3       11798           경제           유통/쇼핑      새 통큰상품 기대하세요…롯데마트 창립17주년 행사   
4       15381           경제            대덕특구              추석 열차 승차권 10~11일 예매   

                                         newsContent  label  
0  “대전산업단지 발전과 인식 변화를 위해 최선의 노력을 다하겠다.” 제14대 대전산업...      1  
1  힐튼 가든 인 서울 강남은 다음달 1일부터 루프톱 야외 수영장을 오픈, 온수풀을 가...      0  
2  이마트가 지속되는 한파에 끝이 보이지 않는 겨울을 이기기 위한 비법으로 겨울 패션 ...      1  
3  롯데마트는 창립 17주년을 맞아 다양한 '통큰 상품'을 새롭게 출시한다고 26일 밝...      1  
4  올해 추석 연휴(9월 9~14일) 열차 승차권(좌석지정 승차권) 예매가 오는 10일...      1  
dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory  \
0       21851           경제              산업   
1        1409           경제     

100%|██████████| 63/63 [00:02<00:00, 25.66it/s]

--- Prediction done ---
 ----- accuracy:60.6% -----
----- f1_score(macro): 37.7% ------
--- Save result ---


,title,cleanBody,clickbaitClass
0,"본그룹 \""일상 속 걷기로 비대면 기부 캠페인 참여해요\""",국내 대표 한식 프랜차이즈 기업 본아이에프가 속한 '본그룹'이 오늘부터 내달 16일...,1
1,"\""그림 완성하고 미니 콘서트도\"" 아이파크몰, 체험형 콘텐츠 강화",HDC아이파크몰은 고객이 직접 참여하고 경험할 수 있는 체험형 콘텐츠를 통해 오프라...,1
2,"LH 미분양아파트 소진 '진땀', 공급실적 올리기위해 알선수수료까지 지원키로",연말을 앞두고 한국토지주택공사(LH) 대전충남지역본부가 아파트 공급실적을 끌어올리기...,1
3,"신한銀, 비대면 '땡겨요 사업자 대출' 출시…최대 1000만원 가능",신한은행은 시중은행 최초로 땡겨요 입점 개인사업자를 위한 신용대출 상품인 '땡겨요 ...,1
4,"[신년사]김현수 농식품부 장관 \""농업, 데이터 기반 첨단산업 변모\""",김현수 농림축산식품부 장관은 농업을 데이터 기반의 첨단산업으로 변모시키고 젊은 인재...,1
5,충청권 지방은행 설립 속도낸다,충청권 지방은행 설립을 위한 대전시와 지역경제계의 발걸음이 빨라질 것으로 보인다.\...,1
6,지역 하반기 고용시장 '쾌청',올해 하반기 대전·충청지역의 고용시장이 활기를 띨 전망이다.\n12일 대전고용노동청...,1
7,"삼성전자, 산학협력으로 모바일 기술 개발·인재 육성 가속",삼성전자가 최첨단 모바일 기술 개발과 인재 육성에 속도를 내기 위해 국내 대학들과의...,1
8,"이영 중기부 장관 후보 \""정치하기 위해 스펙 쌓기·기업 이용 안했다\""","이영 중소벤처기업부 장관 후보자는 11일 \""정치를 하기 위해 스펙 쌓기를 하거나 ...",1
9,지방은행 설립 세종시도 힘 보탠다,대전시가 추진중인 지방은행 부활 문제가 전국적 이슈로 부상하는 가운데 세종시까지 이...,1
